In [1]:
!pip install fuzzymatcher
!pip install tweepy
!pip install pycountry
!pip install nltk
!pip install textblob
!pip install wordcloud
!pip install scikit-learn
!pip install pickle-mixin
!pip install pycountry

In [2]:
import os
import tweepy as tw
import pandas as pd
import time
import sys
import datetime 
from pathlib import Path
import fuzzymatcher
from google.colab import files
import pickle
import nltk
import re
import pycountry

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


# Import word_tokenize and stopwords from nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from textblob import TextBlob


#key and token generated through Twitter Developer module
consumer_key= 'QLdhEgbqKEZCSVsW0kedh4cXi'
consumer_secret= '1SwSssnHayK8c1bdsiDksMhvaztv9rDQ4cva2NM7VTafXQb9Yr'
access_token= '1441295872852066306-mINc6c8y3TwGaUEV8ytUdqkipK6lua'
access_token_secret= 'W6oC0kDfLjucXVPDJm8ZTtrQPxgYqmw5G922SS0SEGGBg'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def get_tweets(search_words):
    auth = tw.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tw.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
    
    
    start_date="2021-01-01"


    # Collect tweets
    tweets = tw.Cursor(api.search,
                           q=search_words,
                           lang="en",
                           since=start_date).items(2000) 
    
    # Collect a list of tweets
    users_locs = [[tweet.user.screen_name, tweet.user.location,tweet.text,tweet.created_at, tweet.id, tweet.id_str, tweet.truncated, tweet.entities, tweet.source, tweet.in_reply_to_status_id, tweet.in_reply_to_status_id_str, tweet.in_reply_to_user_id, tweet.in_reply_to_user_id_str, tweet.in_reply_to_screen_name, tweet.user, tweet.geo,tweet.coordinates, tweet.place, tweet.contributors,tweet.is_quote_status, tweet.retweet_count, tweet.favorite_count, tweet.favorited, tweet.retweeted, tweet.lang] for tweet in tweets]
    users_locs

            
    #making dataframe for the list
    tweet_text = pd.DataFrame(data=users_locs, 
                                columns=[ "user.screen_name",  "user.location", "text", "created_at",  "id",  "id_str",  
                                         "truncated",  "entities",  "source",  "in_reply_to_status_id",  "in_reply_to_status_id_str", 
                                         "in_reply_to_user_id",  "in_reply_to_user_id_str",  "in_reply_to_screen_name", 
                                         "user",  "geo", "coordinates",  "place",  "contributors", "is_quote_status",  
                                         "retweet_count",  "favorite_count",  "favorited",  "retweeted",  "lang"])
    return tweet_text
    




In [7]:
hashtag=input("Enter the hashtag (eg : #wellbeing) =")
WHITE=get_tweets(str(hashtag))
WHITE

Enter the hashtag (eg : #wellbeing) =#wellbeing


,user.screen_name,user.location,text,created_at,id,id_str,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang
0,TujyeKunywa,United States,RT @Zenorafrica: Give the gift of pampering to...,2021-10-16 08:11:07,1449286771452284929,1449286771452284929,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",Tujye kunywa,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,None,None,None,False,7,0,False,False,en
1,rhopcouncil,"Ottawa, Ontario",RT @opheacanada: Being outdoors can make a dif...,2021-10-16 08:10:36,1449286642674634754,1449286642674634754,False,"{'hashtags': [{'text': 'wellbeing', 'indices':...",Twitter Web App,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,None,None,None,False,7,0,False,False,en
2,Flycatc08394457,"Chester-le-Street, England",RT @JuliaBradbury: We are nothing without our ...,2021-10-16 08:10:07,1449286518732992512,1449286518732992512,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",Twitter for Android,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,None,None,None,False,1188,0,False,False,en
3,Arti_Sharma28,London,RT @sophiegarner1: @DrTinarae purchased at th...,2021-10-16 08:10:03,1449286504187060230,1449286504187060230,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",Twitter for iPhone,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,None,None,None,False,6,0,False,False,en
4,peteraf1,EU just round the corner,RT @SVPcharity: Poorer children should be allo...,2021-10-16 08:08:35,1449286135545487364,1449286135545487364,False,"{'hashtags': [{'text': 'school', 'indices': [1...",Twitter Web App,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,None,None,None,False,1,0,False,False,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,MeyfroodtK,"Sint-Niklaas, België",RT @hrm_ob: How can #leadership #wellbeing and...,2021-10-15 09:34:49,1448945447104548878,1448945447104548878,False,"{'hashtags': [{'text': 'leadership', 'indices'...",Twitter Web App,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,None,None,None,True,2,0,False,False,en
1996,Whynicotics,London,RT @keychangesmusic: We're supporting young mu...,2021-10-15 09:34:19,1448945320650477590,1448945320650477590,False,"{'hashtags': [{'text': 'mentalhealth', 'indice...",Twitter Web App,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,None,None,None,False,14,0,False,False,en
1997,greenbeetle_s,París,#wellbeing Wheat Straw Environmentally Friendl...,2021-10-15 09:33:10,1448945030463373312,1448945030463373312,False,"{'hashtags': [{'text': 'wellbeing', 'indices':...",SocialRabbit Plugin,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,None,None,None,False,0,1,False,False,en
1998,General_Medical,Peterborough,Today is #GlobalHandwashingDay and we all know...,2021-10-15 09:33:00,1448944990785245203,1448944990785245203,True,"{'hashtags': [{'text': 'GlobalHandwashingDay',...",Sprout Social,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,None,None,None,False,0,1,False,False,en


In [11]:
# We do not need first two columns. Let's drop them out.
WHITE.drop(columns=["coordinates","place","contributors"], axis=1, inplace=True)
# Drop duplicated rows
WHITE["created_at"] = pd.to_datetime(WHITE["created_at"])
# Print the info again
print(WHITE.info()) #wellbeing

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   user.screen_name           2000 non-null   object        
 1   user.location              2000 non-null   object        
 2   text                       2000 non-null   object        
 3   created_at                 2000 non-null   datetime64[ns]
 4   id                         2000 non-null   int64         
 5   id_str                     2000 non-null   object        
 6   truncated                  2000 non-null   bool          
 7   entities                   2000 non-null   object        
 8   source                     2000 non-null   object        
 9   in_reply_to_status_id      22 non-null     float64       
 10  in_reply_to_status_id_str  22 non-null     object        
 11  in_reply_to_user_id        40 non-null     float64       
 12  in_rep

In [12]:
def process_tweets(tweet):
    
    #remove emoji
    tweet =re.sub( "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+",'', str(tweet),flags=re.MULTILINE) # no emoji
    
    
    # Remove links
    tweet = re.sub(r"http\S+", "", tweet,flags=re.MULTILINE)
    
    # Remove mentions and hashtag
    tweet = re.sub(r'\@\w+|\#','', tweet)
    
    
    
    # Tokenize the words
    tokenized = word_tokenize(tweet)

    # Remove the stop words
    tokenized = [token for token in tokenized if token not in stopwords.words("english")] 

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    tokenized = [lemmatizer.lemmatize(token, pos='a') for token in tokenized]

    # Remove non-alphabetic characters and keep the words contains three or more letters
    tokenized = [token for token in tokenized if token.isalpha() and len(token)>2]
    
    return tokenized
    
# Call the function and store the result into a new column
WHITE["Processed"] = WHITE["text"].str.lower().apply(process_tweets)

# Print the first fifteen rows of Processed
display(WHITE[["Processed"]])



# Get the tweet lengths
WHITE["Length"] = WHITE["text"].str.len()
# Get the number of words in tweets
WHITE["Words"] = WHITE["text"].str.split().str.len()
# Display the new columns
display(WHITE[["Length", "Words"]])


# Fill the missing values with unknown tag
WHITE["user.location"].fillna("unknown", inplace=True)
# Print the unique locations and number of unique locations
"Unique Values:",WHITE["user.location"].unique()
print("Unique Value count:",len(WHITE["user.location"].unique()))

,Processed
0,"[give, gift, pampering, someone, special, toda..."
1,"[outdoors, make, difference, wellbeing, physic..."
2,"[nothing, without, health, prioritise, health,..."
3,"[purchased, years, tes, sen, show, every, teac..."
4,"[poor, children, allowed, garden, school, help..."
...,...
1995,"[leadership, wellbeing, attentivecommunication..."
1996,"[supporting, young, musicians, mcr, struggling..."
1997,"[wellbeing, wheat, straw, environmentally, fri..."
1998,"[today, globalhandwashingday, know, important,..."


,Length,Words
0,140,24
1,143,17
2,148,25
3,140,24
4,139,21
...,...,...
1995,140,15
1996,140,20
1997,119,13
1998,140,22


Unique Value count: 900


In [13]:
def get_countries(location):
    
    # If location is a country name return its alpha2 code
    if pycountry.countries.get(name= location):
        return pycountry.countries.get(name = location).alpha_2
    
    # If location is a subdivisions name return the countries alpha2 code
    try:
        pycountry.subdivisions.lookup(location)
        return pycountry.subdivisions.lookup(location).country_code
    except:
        # If the location is neither country nor subdivision return the "unknown" tag
        return "unknown"

# Call the function and store the country codes in the Country column
WHITE["Country"] = WHITE["user.location"].apply(get_countries)

# Print the unique values
print(WHITE["Country"].unique())

# Print the number of unique values
print("Number of unique values:",len(WHITE["Country"].unique()))

type(WHITE["Processed"])
flat_list = [item for sublist in WHITE["Processed"] for item in sublist]



# Create our contextual stop words
tfidf_stops =["cash"
,"compensation"
,"earnings"
,"interest"
,"livelihood"
,"pay"
,"proceeds"
,"profit"
,"revenue"
,"royalty"
,"salary"
,"wage"
,"assets"
,"avails"
,"benefits"
,"commission"
,"dividends"
,"drawings"
,"gains"
,"gravy"
,"gross"
,"harvest"
,"honorarium"
,"means"
,"payoff"
,"receipts"
,"returns"]




# Initialize a Tf-idf Vectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words= tfidf_stops)
# Fit and transform the vectorizer
tfidf_matrix = vectorizer.fit_transform(flat_list)
# Let's see what we have
display(tfidf_matrix)
# Create a DataFrame for tf-idf vectors and display the first rows
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns= vectorizer.get_feature_names())
tweets_processed=tfidf_df.head()
display(tweets_processed)

tweets_processed=WHITE


print("Since:",tweets_processed["created_at"].min())
# Print the maximum datetime
print("Until",tweets_processed["created_at"].max())


# Print the value counts of Country column
#print(tweets_processed["Country"].value_counts())
# Create a new DataFrame called frequencies
frequencies = pd.DataFrame(tfidf_matrix.sum(axis=0).T,index=vectorizer.get_feature_names(),columns=['total frequency'])
# Display the most 20 frequent words
#display(frequencies.sort_values(by='total frequency',ascending=False))


# Add polarities and subkectivities into the DataFrame by using TextBlob
tweets_processed["Polarity"] = tweets_processed["Processed"].apply(lambda word: TextBlob(str(word)).sentiment.polarity)
tweets_processed["Subjectivity"] = tweets_processed["Processed"].apply(lambda word: TextBlob(str(word)).sentiment.subjectivity)

# Display the Polarity and Subjectivity columns
#display(tweets_processed[["Polarity","Subjectivity"]])

# Define a function to classify polarities
def analyse_polarity(polarity):
    if polarity > 0:
        return "Positive"
    if polarity == 0:
        return "Neutral"
    if polarity < 0:
        return "Negative"

# Apply the funtion on Polarity column and add the results into a new column
tweets_processed["Polarity Scores"] = tweets_processed["Polarity"].apply(analyse_polarity)

# Display the Polarity and Subjectivity Analysis
#display(tweets_processed[["Polarity Scores"]])

# Print the value counts of the Label column
print(tweets_processed["Polarity Scores"].value_counts())
tweets_processed

['US' 'unknown' 'SG' 'AU' 'GB' 'HK' 'MY' 'NZ' 'IE' 'CY' 'LK' 'CA' 'PK'
 'IN' 'PE' 'CO' 'AR' 'ZA' 'MV' 'DE' 'UG' 'KE' 'IL' 'NL' 'CH' 'GI' 'MA'
 'NO' 'FR' 'TT' 'ES']
Number of unique values: 31


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['be', 'first'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<18504x3492 sparse matrix of type '<class 'numpy.float64'>'
	with 18457 stored elements in Compressed Sparse Row format>

,aaa,abandon,abc,aberdeen,aberlady,able,aboriginal,absolutely,abundance,abused,abusing,academia,academic,academy,accelerate,acceptable,accepted,access,accessed,accessible,accommodation,according,accredited,achievable,achieve,achievegoals,achievement,achievements,achievesuccess,acid,acknowledge,acknowledgeassessassist,acne,across,act,acted,actgovernment,acting,action,active,...,worthy,would,wow,wrinkles,wrist,write,writer,writing,writingcommunity,written,wrong,wrote,yanyan,yatton,yea,year,years,yellow,yes,yesterday,yoga,yogalife,yoghurt,york,yorkshire,young,youth,youthmentalhealth,youtube,yrself,zarantonello,zazen,zen,zodiacsigns,zones,zonesofregulation,zoom,𝐁𝐞𝐢𝐧𝐠,𝐇𝐞𝐚𝐥𝐭𝐡,𝐖𝐞𝐥𝐥
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Since: 2021-10-15 09:32:57
Until 2021-10-16 08:11:07
Positive    1165
Neutral      614
Negative     221
Name: Polarity Scores, dtype: int64


,user.screen_name,user.location,text,created_at,id,id_str,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,Processed,Length,Words,Country,Polarity,Subjectivity,Polarity Scores
0,TujyeKunywa,United States,RT @Zenorafrica: Give the gift of pampering to...,2021-10-16 08:11:07,1449286771452284929,1449286771452284929,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",Tujye kunywa,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,False,7,0,False,False,en,"[give, gift, pampering, someone, special, toda...",140,24,US,0.378571,0.485714,Positive
1,rhopcouncil,"Ottawa, Ontario",RT @opheacanada: Being outdoors can make a dif...,2021-10-16 08:10:36,1449286642674634754,1449286642674634754,False,"{'hashtags': [{'text': 'wellbeing', 'indices':...",Twitter Web App,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,False,7,0,False,False,en,"[outdoors, make, difference, wellbeing, physic...",143,17,unknown,0.375000,0.375000,Positive
2,Flycatc08394457,"Chester-le-Street, England",RT @JuliaBradbury: We are nothing without our ...,2021-10-16 08:10:07,1449286518732992512,1449286518732992512,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",Twitter for Android,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,False,1188,0,False,False,en,"[nothing, without, health, prioritise, health,...",148,25,unknown,0.500000,0.888889,Positive
3,Arti_Sharma28,London,RT @sophiegarner1: @DrTinarae purchased at th...,2021-10-16 08:10:03,1449286504187060230,1449286504187060230,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",Twitter for iPhone,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,False,6,0,False,False,en,"[purchased, years, tes, sen, show, every, teac...",140,24,unknown,0.000000,0.000000,Neutral
4,peteraf1,EU just round the corner,RT @SVPcharity: Poorer children should be allo...,2021-10-16 08:08:35,1449286135545487364,1449286135545487364,False,"{'hashtags': [{'text': 'school', 'indices': [1...",Twitter Web App,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,False,1,0,False,False,en,"[poor, children, allowed, garden, school, help...",139,21,unknown,-0.250000,0.400000,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,MeyfroodtK,"Sint-Niklaas, België",RT @hrm_ob: How can #leadership #wellbeing and...,2021-10-15 09:34:49,1448945447104548878,1448945447104548878,False,"{'hashtags': [{'text': 'leadership', 'indices'...",Twitter Web App,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,True,2,0,False,False,en,"[leadership, wellbeing, attentivecommunication...",140,15,unknown,0.000000,0.000000,Neutral
1996,Whynicotics,London,RT @keychangesmusic: We're supporting young mu...,2021-10-15 09:34:19,1448945320650477590,1448945320650477590,False,"{'hashtags': [{'text': 'mentalhealth', 'indice...",Twitter Web App,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,False,14,0,False,False,en,"[supporting, young, musicians, mcr, struggling...",140,20,unknown,0.175000,0.325000,Positive
1997,greenbeetle_s,París,#wellbeing Wheat Straw Environmentally Friendl...,2021-10-15 09:33:10,1448945030463373312,1448945030463373312,False,"{'hashtags': [{'text': 'wellbeing', 'indices':...",SocialRabbit Plugin,NaN,None,NaN,None,None,User(_api=<tweepy.api.API object at 0x7f57040c...,None,False,0,1,False,False,en,"[wellbeing, wheat, straw, environmentally, fri...",119,13,unknown,0.375000,0.500000,Positive
1998,General_Medical,Peterborough,Today is #GlobalHandwashingDay and we all know...,2021-10-15 09:33:00,1448944990785245203,1448944990785245203,True,"{'hashtags': [{'text': 'GlobalHandwashingDay',...",Sprout Social,NaN,None,Na